#### If you are running cohort1_goodresults_edited, DO NOT do anymore filtering. Just run logistic reg

In [1]:
import os
import gc
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import ttest_ind
from statsmodels.sandbox.stats.multicomp import multipletests

In [3]:
path = r'C:\Users\you_l\OneDrive\Desktop\Work stuff\VAP'
os.chdir(path)

In [4]:
pd.set_option('display.max_columns', 200)

In [5]:
#df = pd.read_csv('edited_data_cleaned_updated221129.csv')

In [6]:
df = pd.read_csv('cohort1_goodresults_edited.csv')
sofa_df = pd.read_csv('sofa.csv')
ventduration_df = pd.read_csv('vent_duration.csv')
medical_hist_df = pd.read_csv('personal_medical_hist.csv')
gcs_df = pd.read_csv('gcs.csv')

In [11]:
#df.race = df.race.copy().apply(lambda x: 'White' if x=='White' else 'Non_white')

In [15]:
#print(f'Original number of stayids = {df.stay_id.nunique()}')
#df = df[df.age < 90]
#print(f'Number of stayids after age exclusion = {df.stay_id.nunique()}')

Original number of stayids = 9372
Number of stayids after age exclusion = 9079


In [7]:
# This code is to match the descriptive stats notebook
df['icu_los'] = pd.to_datetime(df['outtime']) - pd.to_datetime(df['intime'])
df['icu_los'] = np.round(df['icu_los'].dt.total_seconds()/3600)

df = df.merge(sofa_df, on='stay_id', how='left')
df = df.merge(ventduration_df, on='stay_id', how='left')
df = df.merge(medical_hist_df, on='subject_id', how='left')
df = df.merge(gcs_df, on='stay_id', how='left')

# Impute
df[['hist_alcohol', 'host_smoking', 'hist_lungdisease','hist_cancer']] = df[['hist_alcohol', 'host_smoking', 'hist_lungdisease','hist_cancer']].fillna(0)
df[['hist_alcohol', 'host_smoking', 'hist_lungdisease','hist_cancer']] = df[['hist_alcohol', 'host_smoking', 'hist_lungdisease','hist_cancer']].astype(int)

for i in ['gcs_motor', 'gcs_verbal', 'gcs_eyes', 'gcs_unable']:
    x = df[i].mode()
    df[i] = df[i].fillna(x[0])

In [8]:
def gcs_categories(x):
    if x>=3 and x<=7:
        return '3-7'
    elif x>=8 and x<=13:
        return '8-13'
    else:
        return '14-15'
    
def sofa_categories(x):
    if x <=6:
        return '0-6'
    elif x>=7 and x<=9:
        return '7-9'
    else:
        return '>9'

In [9]:
df['sofa'] = df['sofa'].apply(sofa_categories)
df['gcs'] = df['gcs'].apply(gcs_categories)

In [10]:
df['cuff_plus_richmond'] = df['cuffpressure_yn'] + df['richmond_yn']
df['cuff_plus_richmond'] = df['cuff_plus_richmond'].apply(lambda x: '2' if x==2 else '0-1')

In [11]:
# Remove cuffpressure_yn from vent_bundle_score
df['vent_bundle_score'] = df['vent_bundle_score'] - df['cuffpressure_yn']

Perform analysis on those without any readmissions

In [13]:
#df = df.sort_values(by=['subject_id', 'intime', 'race'], ascending=[True, True, False]
#                   ).drop_duplicates(subset=['subject_id'], keep='last')

In [7]:
print(df.subject_id.nunique())
print(df.hadm_id.nunique())
print(df.stay_id.nunique())
print(len(df))

7935
8730
9079
9893


# 2 Logistic Regression

In [13]:
treatments = ['vent_bundle_score', 'vap_bundle_score', 'cuffpressure_yn', 'headup_yn', 'richmond_yn',
              'dvtprophy_yn', 'Chlorhex_yn', 'gastroprophylaxis_yn', 'cuff_plus_richmond']

other_treatments = ['hemoglobin_0hrs_value', 'hemoglobin_24hrs_value', 'max_ett_size',
                    'min_ett_size', 'avg_total_peep', 'avg_tv_observed', 'freq_oral_care', 'freq_subglottal_suct',
                    'riker_sas_24hrs_value', 'cont_neb_med_present','humid_hme', 'humid_active', 'most_freq_vent_mode', 'most_freq_gagreflex_value',
                    'most_freq_head_of_bed_orientation', 'richmond_target', 'most_freq_cough_effort', 'most_freq_cough_type',
                    'oral_care_provided']

outcome = ['vap_diagnosed']

covariates = ['age', 'charlson_comorbidity_index', 'race', 'is_female', 'admission_type', 'first_careunit', 'sofa',
             'host_smoking', 'hist_lungdisease', 'gcs']

## 2.1 Test Significance of covariates/confounder using univariable analysis

In [14]:
def age_categories(x):
    if x<45:
        return '<45'
    elif x>=45 and x<=64:
        return '45-64'
    elif x>=65 and x<=74:
        return '65-74'
    elif x>=75 and x<=84:
        return '75-84'
    else:
        return '>84'
    
def cci_categories(x):
    if x==0:
        return '0'
    elif x==1 or x==2:
        return '1-2'
    elif x==3 or x==4:
        return '3-4'
    else:
        return '>4'

In [15]:
df['age'] = df['age'].apply(age_categories)
df['charlson_comorbidity_index'] = df['charlson_comorbidity_index'].apply(cci_categories)

In [16]:
def evaluate_covariates(df, covariates, outcome):
    y = df[outcome]
    df = pd.get_dummies(df[covariates])
    cov_cols = df.columns.tolist()
    model_out = []
    for i in cov_cols:
        X = df[i]
        X = sm.add_constant(X)
        #cols = X.columns[1:]
        logit = sm.Logit(y, X)
        results = logit.fit(maxiter=1000)
        
        ## Odds Ratio
        #params = results.params
        #conf = results.conf_int()
        #conf['Odds Ratio'] = params
        #conf.columns = ['Odds Ratio', '0.025', '0.975']
        #conf = np.round(np.exp(conf), 3)
        
        model_results = {}
        model_results.update({i: {'coeff': np.round(results.params[1],3),
                                        'p-value':np.round(results.pvalues[1],3)
                                       }
                             })
        #model_results.update({cols[j]: {'coeff': np.round(results.params[j+1],3),
        #                                'p-value':np.round(results.pvalues[j+1],3)} for j in range(len(cols))})
        model_results = pd.DataFrame(model_results).T
        #model_results = pd.concat([model_results, pd.DataFrame(conf.iloc[j+1] for j in range(len(cols)))], axis=1)
        model_results.columns = pd.MultiIndex.from_product([['VAP diagnosed ~ Covariate/Confounders'], model_results.columns])
        model_out.append(model_results)
    
    model_out = pd.concat(model_out)
    #display(model_out)
    return model_out

In [17]:
evaluate_covariates(df, covariates, 'vap_diagnosed')

Optimization terminated successfully.
         Current function value: 0.439542
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440598
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440571
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440532
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440625
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440325
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440454
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440586
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440626
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.439668
  

VAP diagnosed ~ Covariate/Confounders  \
                                                                  coeff   
is_female                                                        -0.260   
host_smoking                                                      0.049   
hist_lungdisease                                                 -0.189   
age_45-64                                                         0.079   
age_65-74                                                        -0.022   
age_75-84                                                        -0.174   
age_<45                                                           0.144   
age_>84                                                          -0.115   
charlson_comorbidity_index_1-2                                    0.025   
charlson_comorbidity_index_3-4                                    0.285   
charlson_comorbidity_index_>4                                    -0.227   
race_Non_white                                                    0.191   
race_White                                                       -0.191   
admission_type_Elective                                          -0.096   
admission_type_Emergency                                          0.096   
first_careunit_Combined MICU/SICU                                -0.267   
first_careunit_MICU                                              -0.046   
first_careunit_SICU                                               0.181   
sofa_0-6                                                          0.285   
sofa_7-9                                                         -0.330   
sofa_>9                                                          -0.108   
gcs_14-15                                                        -0.012   
gcs_3-7                                                          -0.107   
gcs_8-13                                                          0.073   

                                           
                                  p-value  
is_female                           0.000  
host_smoking                        0.456  
hist_lungdisease                    0.328  
age_45-64                           0.200  
age_65-74                           0.752  
age_75-84                           0.026  
age_<45                             0.084  
age_>84                             0.392  
charlson_comorbidity_index_1-2      0.774  
charlson_comorbidity_index_3-4      0.000  
charlson_comorbidity_index_>4       0.000  
race_Non_white                      0.002  
race_White                          0.002  
admission_type_Elective             0.390  
admission_type_Emergency            0.390  
first_careunit_Combined MICU/SICU   0.002  
first_careunit_MICU                 0.499  
first_careunit_SICU                 0.003  
sofa_0-6                            0.077  
sofa_7-9                            0.071  
sofa_>9                             0.742  
gcs_14-15                           0.904  
gcs_3-7                             0.520  
gcs_8-13                            0.532

Admission_type isn't significant. We'll drop it from the covariates.

In [18]:
covariates = ['age', 'charlson_comorbidity_index', 'race', 'is_female', 'first_careunit', 'sofa',
             'host_smoking', 'hist_lungdisease', 'gcs']

# 2.1 Try Logistic Regression for ALL interventions and outcome without (uni) and with (multi) covariates 

In [19]:
def evaluate(df, interventions_lst, outcome, covariates_lst):
    df_interventions, df_covar = pd.get_dummies(df[interventions_lst]), pd.get_dummies(df[covariates_lst], drop_first=True)
    interventions_cols = df_interventions.columns.tolist()
    cov_cols = df_covar.columns.tolist()
    y = df[outcome]
    final_out = []
    for i in interventions_cols:
        model_out = []
        
        # Model 1
        X = df_interventions[i]
        X = sm.add_constant(X)
        #cols = X.columns[1:]
        logit = sm.Logit(y, X)
        results = logit.fit(maxiter=1000)
        
        ## Model 1: Odds Ratio
        params = results.params
        conf = results.conf_int()
        conf.insert(0, 'Odds Ratio', params)
        conf.columns = ['Odds Ratio', '0.025', '0.975']
        conf = np.round(np.exp(conf), 3)
        
        model_results = {}
        model_results.update({i: {'coeff': np.round(results.params[1],3),
                                  'p-value':np.round(results.pvalues[1],3)}})
        #model_results.update({cols[j]: {'coeff': np.round(results.params[j+1],3),
        #                                'p-value':np.round(results.pvalues[j+1],3)} for j in range(len(cols))})
        model_results = pd.DataFrame(model_results).T
        model_results = pd.concat([model_results, pd.DataFrame(conf.iloc[1]).T], axis=1)
        model_results.columns = pd.MultiIndex.from_product([['VAP diagnosed ~ Intervention'], model_results.columns])
        model_out.append(model_results)
    
        # Model 2
        #df_covariates = pd.get_dummies(df[covariates], drop_first=True)
        X = pd.concat([X, df_covar], axis=1)
        logit = sm.Logit(y, X)
        results = logit.fit(maxiter=1000)
        
        params = results.params
        conf = results.conf_int()
        conf.insert(0, 'Odds Ratio', params)
        conf.columns = ['Odds Ratio', '0.025', '0.975']
        conf = np.round(np.exp(conf), 3)
        
        model_results = {}
        model_results.update({i: {'coeff': np.round(results.params[1],3),
                                  'p-value': np.round(results.pvalues[1],3)}})
        model_results = pd.DataFrame(model_results).T
        model_results = pd.concat([model_results, pd.DataFrame(conf.iloc[1]).T], axis=1)
        model_results.columns = pd.MultiIndex.from_product([['VAP diagnosed ~ Intervention + covariates/confounders'], model_results.columns])
        model_out.append(model_results)
        
        final_out.append(pd.concat(model_out, axis=1))
    
    final_out = pd.concat(final_out)
    display(final_out)
    return final_out

In [20]:
final_out = evaluate(df, treatments + other_treatments, 'vap_diagnosed', covariates)

Optimization terminated successfully.
         Current function value: 0.439909
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.436321
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440352
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.436654
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440305
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.436537
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.440421
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.436767
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.439654
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.436011
  

VAP diagnosed ~ Intervention  \
                                                                        coeff   
vent_bundle_score                                                      -0.116   
vap_bundle_score                                                       -0.081   
cuffpressure_yn                                                         0.181   
headup_yn                                                              -0.175   
richmond_yn                                                            -0.291   
dvtprophy_yn                                                           -0.021   
Chlorhex_yn                                                            -0.045   
gastroprophylaxis_yn                                                   -0.534   
hemoglobin_0hrs_value                                                   0.064   
hemoglobin_24hrs_value                                                  0.055   
max_ett_size                                                            0.385   
min_ett_size                                                            0.318   
avg_total_peep                                                          0.020   
avg_tv_observed                                                        -0.000   
freq_oral_care                                                          0.009   
freq_subglottal_suct                                                    0.010   
riker_sas_24hrs_value                                                  -0.074   
cont_neb_med_present                                                    0.605   
humid_hme                                                              -0.150   
humid_active                                                            0.146   
cuff_plus_richmond_0-1                                                  0.231   
cuff_plus_richmond_2                                                   -0.231   
most_freq_vent_mode_APRV                                                0.099   
most_freq_vent_mode_CMV                                                -0.032   
most_freq_vent_mode_CPAP                                               -0.049   
most_freq_vent_mode_NIV                                                -0.384   
most_freq_vent_mode_PSV                                                 0.060   
most_freq_gagreflex_value_Impaired                                      0.262   
most_freq_gagreflex_value_Intact                                       -0.262   
most_freq_head_of_bed_orientation_< 45 Degrees                         -0.168   
most_freq_head_of_bed_orientation_>= 45 Degrees                        -0.009   
most_freq_head_of_bed_orientation_Reverse T-berg                        0.314   
richmond_target_0 to -1                                                -0.291   
richmond_target_<-1                                                     0.232   
richmond_target_>0                                                      0.055   
most_freq_cough_effort_Impaired                                         0.001   
most_freq_cough_effort_Strong                                          -0.001   
most_freq_cough_type_Non-productive                                    -0.088   
most_freq_cough_type_Productive                                         0.088   
oral_care_provided_None                                                -0.544   
oral_care_provided_VAP Kit                                              0.162   
oral_care_provided_oral care                                           -0.159   

                                                                            \
                                                 p-value Odds Ratio  0.025   
vent_bundle_score                                  0.001      0.891  0.834   
vap_bundle_score                                   0.031      0.922  0.856   
cuffpressure_yn                                    0.022      1.199  1.027   
headup_yn                                          0.059      0.839  0.700   
richmond_yn     

## 2.1.1 Bonferroni

In [19]:
#p_vals_base = final_out[('VAP diagnosed ~ Intervention', 'p-value')].tolist()
#p_vals_cov = final_out[('VAP diagnosed ~ Intervention + covariates/confounders', 'p-value')].tolist()
#p_adjusted_base = multipletests(p_vals_base, alpha=0.1, method='bonferroni')
#p_adjusted_cov = multipletests(p_vals_cov, alpha=0.1, method='bonferroni')
#final_out[('VAP diagnosed ~ Intervention', 'p-value')] = p_adjusted_base[1]
#final_out[('VAP diagnosed ~ Intervention + covariates/confounders', 'p-value')] = p_adjusted_cov[1]

In [ ]:
#final_out

## 2.2.1 Dealing with Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X = df[significant_treatments]
X = pd.get_dummies(X, drop_first=True)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

In [ ]:
vif_data

In [ ]:
# Drop hemoglobin_24hrs, avg_total_peep (Doesn't work)
# X = X.drop(['hemoglobin_24hrs_value', 'avg_total_peep'], axis=1)

# Drop hemoglobin_0hrs, min_ett_size
#X = X.drop(['hemoglobin_0hrs_value', 'min_ett_size'], axis=1)

X = X.drop(['hemoglobin_0hrs_value', 'hemoglobin_24hrs_value','min_ett_size', 'avg_total_peep'], axis=1)

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif_data

## 2.2.2 Re-run multivariable logreg after dropping those high VIF features

In [ ]:
# Try multivariable again after removing these high collinear ones
X = df[significant_treatments]
X = pd.get_dummies(X, drop_first=True)
X = X.drop(['hemoglobin_0hrs_value', 'hemoglobin_24hrs_value','min_ett_size', 'avg_total_peep'], axis=1)
X = pd.concat([X, df[covariates]], axis=1)
X = pd.get_dummies(X, drop_first=True)  #I do this so i can push all covariates variable to the back.
X = sm.add_constant(X)
y = df['vap_diagnosed']
logit = sm.Logit(y, X)
results = logit.fit(maxiter=1000)
results.summary()

In [ ]:
# Get odds ratio:
params = results.params
conf = results.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['Odds Ratio', '0.025', '0.975']
conf = np.round(np.exp(conf), 3)
conf